In [123]:
import pandas as pd
import os
from datetime import datetime
import time

In [124]:
# START SCRIPT TIMER
print(f"Start Time: {datetime.now()}")
tic = time.perf_counter()

Start Time: 2024-04-18 13:49:33.582194


In [115]:
# ADD USER FILE PATHS

# BEDDN spatial file used to create generate near table
dunslocs_file = r'Z:\UHC_Data\NETS_UHC\NETS2022\Geodatabases\NETS2022_locs.gdb\DunsLocations20231130_v2'

# participant location file used to create generate near table
mesa_locs_file = r'X:\AddressGeocoding\From_MESA_Air\Data\MESAAIR_locs.gdb\mesa_locs_aeac_zcta10'

# generate near table
near_table_file = r'X:\AddressGeocoding\From_MESA_Air\Data\Temp\scratch\NETS_linkage_test.gdb\mesa_nets_linkage_test_sample_mini'

classifiedlong_file = r'Z:\UHC_Data\NETS_UHC\NETS2022\Data\Final\ClassifiedLong20231127.txt'
dunsmove_file = r'Z:\UHC_Data\NETS_UHC\NETS2022\Data\Final\DunsMove20231201.txt'
cat_descriptions_file = r'Z:\UHC_Data\NETS_UHC\NETS2022\Data\Final\CategoryDescriptions20231127.txt'
xwalk_file = r'Z:\UHC_Data\NETS_UHC\NETS2022\Data\Final\BG_CC_TC_Xwalk20231023.txt'
output_folder = r'D:\scratch'

In [116]:
# USER INPUTS

# provide list of year(s)
years = [2000, 2005]

# use hierarchy? True or False
hierarchy = True

# limit categories? True or False
limit_cats = False

# if True, pick categories:

# print list of domains
desc = pd.read_csv(cat_descriptions_file, sep='\t')
domlist = list(desc['Domain'].unique())
print(domlist)

# provide list of categories by entire domain (optional):
domains = ['Financial']

# provide list of individual categories (optional):
categories = ['DLR', 'CMN', 'GRY']

# provide UHCMatchCodeRank threshold (<=) for NETS/BEDDN and participant location. if no threshold desired, use value of 99:
UHCMCR_NETS = 6
UHCMCR_PART = 6

# provide buffer sizes in km
buff_sizes = [0.5, 1, 1.609]


['Food', 'Healthcare', 'Physical Activity', 'Social', 'Cognitive Enrichment', 'Financial', 'Alcohol, Tobacco, Marijuana, Firearm', 'Walking', 'Transportation', 'Disaster/Construction']


In [117]:
arcpy.env.workspace = r"F:\Arc_Projects\NETS_test_linkage"

In [118]:
# CONVERT FC/GDB TABLE TO PANDAS DF

# define function to convert fc table to pandas dataframe
def table_to_data_frame(in_table, input_fields=None, where_clause=None):
    """Function will convert an arcgis table into a pandas dataframe with an object ID index, and the selected
    input fields using an arcpy.da.SearchCursor."""
    OIDFieldName = arcpy.Describe(in_table).OIDFieldName
    if input_fields:
        final_fields = [OIDFieldName] + input_fields
    else:
        final_fields = [field.name for field in arcpy.ListFields(in_table)]
    data = [row for row in arcpy.da.SearchCursor(in_table, final_fields, where_clause=where_clause)]
    fc_dataframe = pd.DataFrame(data, columns=final_fields)
    fc_dataframe = fc_dataframe.set_index(OIDFieldName, drop=True)
    return fc_dataframe

In [125]:
# LOAD NEAR TABLE AND MESA LOCS THEN MERGE

# load table and round NEAR_DIST to 3 decimal points
near_df = table_to_data_frame(near_table_file, input_fields=['IN_FID', 'NEAR_FID', 'NEAR_DIST'])
near_df['NEAR_DIST'] = near_df['NEAR_DIST'].round(3)

mesa_locs = table_to_data_frame(mesa_locs_file, input_fields=['LOCID_DREXEL', 'UHCMatchCodeRank'])
print(f'nrows of mesa_locs before removing low quality geocodes: {len(mesa_locs)}')

# remove records where UHCMatchCodeRank is above threshold
mesa_locs = mesa_locs.loc[mesa_locs['UHCMatchCodeRank'] <= UHCMCR_PART]
print(f'nrows of mesa_locs after removing low quality geocodes: {len(mesa_locs)}')

# merge participant location unique ids and uhcmatchcoderank to near table
join_mesa = (near_df
              .merge(mesa_locs, how='left', left_on='IN_FID', right_on='OBJECTID')
              .drop(columns=['IN_FID', 'UHCMatchCodeRank'])
             )
del mesa_locs, near_df

nrows of mesa_locs before removing low quality geocodes: 16703
nrows of mesa_locs after removing low quality geocodes: 15160


In [126]:
# LOAD DUNS LOCS THEN MERGE
duns_locs = table_to_data_frame(dunslocs_file, input_fields=['AddressID', 'UHCMatchCodeRank'])
print(f'nrows of duns_locs before removing low quality geocodes: {len(duns_locs)}')

# remove records where UHCMatchCodeRank is above threshold
duns_locs = duns_locs.loc[duns_locs['UHCMatchCodeRank'] <= UHCMCR_NETS]
print(f'nrows of duns_locs after removing low quality geocodes: {len(duns_locs)}')

# merge beddn addressids and uhcmatchcoderank
join_addressid = (join_mesa
                  .merge(duns_locs, how='left', left_on='NEAR_FID', right_on='OBJECTID')
                  .drop(columns=['NEAR_FID', 'UHCMatchCodeRank'])
                 )
del join_mesa # add duns_locs here

nrows of duns_locs after removing low quality geocodes: 25025336


In [128]:
join_addressid

,NEAR_DIST,LOCID_DREXEL,AddressID
0,0.015,83870937.0,A014454029
1,0.024,83870937.0,A009584402
2,0.024,83870937.0,A011312061
3,0.025,83870937.0,A016498116
4,0.025,83870937.0,A023517595
...,...,...,...
999995,4.461,18761288.0,A006182179
999996,4.461,18761288.0,A014215988
999997,4.461,18761288.0,A023648477
999998,4.461,18761288.0,A001494850


In [129]:
# how many unique LOCID_DREXELs are there?
uniq_locid = join_addressid['LOCID_DREXEL'].nunique()
print(f'n of unique LOCID_DREXELs: {uniq_locid}')

# how many unique AddressIDs are there?
uniq_add = join_addressid['AddressID'].nunique()
print(f'n of unique AddressIDs: {uniq_add}')

n of unique LOCID_DREXELs: 6
n of unique AddressIDs: 425840


In [108]:
# READ IN DUNSMOVE AND SUBSET BY YEAR
dunsmove = pd.read_csv(dunsmove_file, sep='\t', usecols=['DunsYear', 'DunsMove', 'AddressID', 'Year'], dtype={'Year':int})

# subset dunsmove for years requested
dunsmove = dunsmove.loc[dunsmove['Year'].isin(years)] 

# merge in dunsmove columns
join_dunsmove = join_addressid.merge(dunsmove, how='left', on='AddressID')
del dunsmove, join_addressid

In [ ]:
# READ IN CLASSIFIED LONG AND SUBSET BY CATEGORY IF APPLICABLE
classlong = pd.read_csv(classifiedlong_file, sep='\t', usecols=['DunsYear','BaseGroup'])

# subset for provided categories
if limit_cats == True:
    # grab all categories in chosen domain(s)
    domain_cats = desc['Category'].loc[desc['Domain'].isin(domains)]  
    all_cats = list(domain_cats)

    # grab all additional categories
    [all_cats.append(category) for category in categories]
    classlong = classlong.loc[classlong['BaseGroup'].isin(all_cats)]
else:
    pass

# del classlong

In [52]:
# MERGE BASE GROUP (CLASSIFIED) DATA 
join_classlong = join_dunsmove.merge(classlong, how='inner', on='DunsYear')

In [53]:
join_classlong.shape

(15802414, 9)

In [54]:
# MERGE HIGH LEVEL CATEGORIES
xwalk = pd.read_csv(xwalk_file, sep='\t')
join_xwalk = join_classlong.merge(xwalk[['BaseGroup', 'HighLevel']], on='BaseGroup')

In [58]:
# APPLY HIERARCHY IF APPLICABLE
if hierarchy == True:
    # join hierarchy
    join_classlong = (join_classlong
                   .merge(desc[['Category', 'Hierarchy']], left_on='BaseGroup', right_on='Category')
                   .drop(columns=['Category']))
    
    # sort by hierarchy, then drop all duplicates of dunsyear, keep first instance
    hierarchy_version = (join_classlong
                         .sort_values(by='Hierarchy')
                         .drop_duplicates(subset=['DunsYear', 'Year'], keep='first'))
else: 
    pass


In [ ]:
# SEPARATE INTO SEPARATE BUFFER BINS



In [59]:
# LOOP THROUGH YEARS TO SPLIT INTO SEPARATE YEAR FILES 
for year in years:
    print(f'working on: {year}')
    temp = measures.loc[measures['Year'] == year]
    output_file = os.path.join(output_folder, f'beddn_businesslevel_measures{year}.txt')
    temp.to_csv(output_file, sep='\t', index=False)

working on: 2000
working on: 2005


In [49]:
# END SCRIPT TIMER 
toc = time.perf_counter()
t = toc - tic
print(f'total time: {round(t/60, 2)} minutes')

total time: 23.86 minutes


In [60]:
join_classlong.shape

(15802414, 10)

In [61]:
temp.head()

,NEAR_DIST,LOCID_DREXEL,UHCMatchCodeRank_MESA,AddressID,UHCMatchCodeRank_NETS,DunsYear,DunsMove,Year,BaseGroup,Hierarchy
9,0.872865,83870937,2,A003385323,1.0,138610097_2005,1.013861e+10,2005.0,TAX,108.0
10,8.037194,35486391,2,A003385323,1.0,138610097_2005,1.013861e+10,2005.0,TAX,108.0
11,5.322168,9105744,2,A003385323,1.0,138610097_2005,1.013861e+10,2005.0,TAX,108.0
12,6.325217,13809317,2,A003385323,1.0,138610097_2005,1.013861e+10,2005.0,TAX,108.0
13,7.428676,88663423,2,A003385323,1.0,138610097_2005,1.013861e+10,2005.0,TAX,108.0


In [ ]:
# add user input for matchcoderank threshold for mesa and nets
# round NEAR_DIST to 3 decimal points and split into bins
# set year to integer?
# make output only participant id, year, basegroup, highlevel (wide?)